In [1]:
import pandas as pd
import requests, json
from datetime import date, timedelta, time, datetime
from collections import defaultdict
import xml.etree.ElementTree as ET
from math import dist

In [2]:
sample_df_humid = pd.read_csv("final_df_effective_humid_before_label.csv",low_memory=False)
sample_df_weather = pd.read_csv("final_weather_his.csv",low_memory=False)
wildfire_his = pd.read_csv("wildfire_his.csv")

In [3]:
# 먼저 2004년부터 2011년까지의 사건 발생 목록 추출
mask = wildfire_his['발생일시'] > 200401010000
temp = wildfire_his.loc[mask]
mask = wildfire_his['발생일시'] < 201112312359
small_his = temp.loc[mask]
small_his

,Unnamed: 0,_id,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
901,901,62be5e0852482b6dab1321a2,201106211707,화,201106211930,223,경남,경남,창녕,남지,...,31.9,54.9,41.0,0.0,7.6,4.7,S,S,1,30.0
902,902,62be5e0852482b6dab1321a3,201106211440,화,201106211730,250,충북,충북,옥천,동이,...,30.0,56.4,38.0,0.0,2.1,3.0,WSW,NNW,3,3.0
903,903,62be5e0852482b6dab1321a4,201106211300,화,201106211440,140,경남,경남,함안,법수,...,29.8,59.2,43.0,0.0,3.0,2.5,E,SSW,10,2.0
904,904,62be5e0852482b6dab1321a5,201106201445,월,201106201845,400,경남,경남,산청,신등,...,31.3,55.7,36.0,0.0,4.6,2.1,E,NE,9,3.0
905,905,62be5e0852482b6dab1321a6,201106201315,월,201106201414,59,경기,경기,여주,산북,...,31.8,54.8,41.0,0.0,4.4,2.3,S,W,5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004,4004,62be5e0852482b6dab132dc1,200401071410,수,200401071420,10,경남,경남,합천,대양,...,10.8,0.0,23.0,0.0,0.0,4.0,,NNW,32,9.5
4005,4005,62be5e0852482b6dab132dc2,200401051528,월,200401051528,0,서울,서울,관악,신림,...,7.8,0.0,40.0,0.0,0.0,1.3,,W,3,3.0
4006,4006,62be5e0852482b6dab132dc3,200401031505,토,200401031555,50,경북,경북,울진,울진,...,8.5,0.0,35.0,0.0,0.0,3.5,,ESE,28,9.0
4007,4007,62be5e0852482b6dab132dc4,200401031135,토,200401031205,30,대구,대구,동,둔산,...,7.3,0.0,36.0,0.0,0.0,2.5,,W,23,1.0


In [4]:
occur_place_his = small_his[["발생일시","발생장소(시도)", "발생장소(시군구)"]]
occur_place_his

,발생일시,발생장소(시도),발생장소(시군구)
901,201106211707,경남,창녕
902,201106211440,충북,옥천
903,201106211300,경남,함안
904,201106201445,경남,산청
905,201106201315,경기,여주
...,...,...,...
4004,200401071410,경남,합천
4005,200401051528,서울,관악
4006,200401031505,경북,울진
4007,200401031135,대구,동


In [5]:
lat_df = pd.read_csv('latitude.csv')
lat_df

,Unnamed: 0,_id,지점번호,지점명,위도,경도
0,0,62be6fb4a82199005252108d,90,속초,38.25085,128.56473
1,1,62be6fb4a82199005252108e,93,북춘천,37.94738,127.75443
2,2,62be6fb4a82199005252108f,95,철원,38.14787,127.30420
3,3,62be6fb4a821990052521090,98,동두천,37.90188,127.06070
4,4,62be6fb4a821990052521091,99,파주,37.88589,126.76648
...,...,...,...,...,...,...
90,90,62be6fb4a8219900525210e7,285,합천,35.56505,128.16994
91,91,62be6fb4a8219900525210e8,288,밀양,35.49147,128.74413
92,92,62be6fb4a8219900525210e9,289,산청,35.41300,127.87910
93,93,62be6fb4a8219900525210ea,294,거제,34.88818,128.60458


In [6]:
# 지점번호 리스트에 없는 지점 위도 경도 찾기
KAKAO_API_KEY = "da33e829f9b3cce36b4dd72eac9312d6"
KAKAO_MAP_URL = "https://dapi.kakao.com/v2/local/search/address.json?query="
def get_lat(place):
    headers = {
        "Authorization" : "KakaoAK {}".format(KAKAO_API_KEY)
    }
    returned_json=json.loads(str(requests.get(KAKAO_MAP_URL+place,headers=headers).text))
    address = returned_json['documents'][0]['address']
    lat_d = {"lat" : str(address['y']), "lng" : str(address['x'])}
    return lat_d

In [7]:
# 중복제거
occur_place_his=occur_place_his.drop_duplicates()
occur_place_his

,발생일시,발생장소(시도),발생장소(시군구)
901,201106211707,경남,창녕
902,201106211440,충북,옥천
903,201106211300,경남,함안
904,201106201445,경남,산청
905,201106201315,경기,여주
...,...,...,...
4004,200401071410,경남,합천
4005,200401051528,서울,관악
4006,200401031505,경북,울진
4007,200401031135,대구,동


In [8]:
occur_place_his['지점명']=''
occur_place_his

C:\Users\yujeyeong\AppData\Local\Temp\ipykernel_53040\274101844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  occur_place_his['지점명']=''


,발생일시,발생장소(시도),발생장소(시군구),지점명
901,201106211707,경남,창녕,
902,201106211440,충북,옥천,
903,201106211300,경남,함안,
904,201106201445,경남,산청,
905,201106201315,경기,여주,
...,...,...,...,...
4004,200401071410,경남,합천,
4005,200401051528,서울,관악,
4006,200401031505,경북,울진,
4007,200401031135,대구,동,


In [9]:
# 지점번호 위도 경도 딕셔너리로 변환
point_dict={}
lat_df['지점명']=lat_df['지점명'].map(lambda x: x[:2] if len(x)>2 and (x[2]=='시' or x[2]=='군') else x)
lat_df=lat_df[["지점번호", "지점명","위도","경도"]]
for i in range(len(lat_df)):
    point_dict[lat_df.iloc[i][1]] = [lat_df.iloc[i][0], lat_df.iloc[i][2],lat_df.iloc[i][3]]
point_dict

{'속초': [90, 38.25085, 128.56473],
 '북춘천': [93, 37.94738, 127.75443],
 '철원': [95, 38.14787, 127.3042],
 '동두천': [98, 37.90188, 127.0607],
 '파주': [99, 37.88589, 126.76648],
 '대관령': [100, 37.67713, 128.71834],
 '춘천': [101, 37.90262, 127.7357],
 '백령도': [102, 37.97396, 124.71237],
 '북강릉': [104, 37.80456, 128.85535],
 '강릉': [105, 37.75147, 128.89099],
 '동해': [106, 37.50709, 129.12433],
 '서울': [108, 37.57142, 126.9658],
 '인천': [112, 37.47772, 126.6249],
 '원주': [114, 37.33749, 127.94659],
 '울릉도': [115, 37.48129, 130.89864],
 '수원': [119, 37.25746, 126.983],
 '영월': [121, 37.18126, 128.45743],
 '충주': [127, 36.97045, 127.9525],
 '서산': [129, 36.77658, 126.4939],
 '울진': [130, 36.99176, 129.41278],
 '청주': [131, 36.63924, 127.44066],
 '대전': [133, 36.37198, 127.37211],
 '추풍령': [135, 36.22025, 127.99458],
 '안동': [136, 36.57293, 128.70734],
 '상주': [137, 36.40837, 128.15741],
 '포항': [138, 36.03201, 129.38002],
 '군산': [140, 36.0053, 126.76135],
 '대구': [143, 35.87797, 128.65295],
 '전주': [146, 35.84092, 127.1

In [10]:
def close_or_exact_place(big_city, small_city):
    if big_city in point_dict.keys():
        return big_city
    elif small_city in point_dict.keys():
        return small_city
    else:
        try:
            point_lat=get_lat(big_city +' '+ small_city+'구')
            for_compare_distance = [] 
            for k in point_dict.keys():
                for_compare_distance.append([dist((float(point_lat['lat']),float(point_lat['lng'])),(point_dict[k][1],point_dict[k][2])), k])
            return sorted(for_compare_distance, key=lambda x: x[0])[0][1]
        except:
            try:
                point_lat=get_lat(big_city +' '+ small_city+'군')
                for_compare_distance = [] 
                for k in point_dict.keys():
                    for_compare_distance.append([dist((float(point_lat['lat']),float(point_lat['lng'])),(point_dict[k][1],point_dict[k][2])), k])
                
                return sorted(for_compare_distance, key=lambda x: x[0])[0][1]
            except:
                try:
                    point_lat=get_lat(big_city +' '+ small_city+'시')
                    for_compare_distance = [] 
                    for k in point_dict.keys():
                        for_compare_distance.append([dist((float(point_lat['lat']),float(point_lat['lng'])),(point_dict[k][1],point_dict[k][2])), k])
                    return sorted(for_compare_distance, key=lambda x: x[0])[0][1]
                except:
                    return small_city

In [11]:
occur_place_his['지점명']=occur_place_his.apply(lambda x: close_or_exact_place(x['발생장소(시도)'], x['발생장소(시군구)']), axis=1)

C:\Users\yujeyeong\AppData\Local\Temp\ipykernel_53040\3760189309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  occur_place_his['지점명']=occur_place_his.apply(lambda x: close_or_exact_place(x['발생장소(시도)'], x['발생장소(시군구)']), axis=1)


In [12]:
# 시간까지 발생시간 자르기
sample_df_humid['tm'] = sample_df_humid['tm'].map(lambda x: x[:-3].replace(' ','').replace('-',''))

#sample_hm_merged['label']=sample_hm_merged['tm'].map(lambda x:x[:-2])
sample_df_humid

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,tm,rnum,stnId,stnNm,ta,taQcflg,rn,...,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te,실효습도
0,7104,7104,0,2004010100,1,90,속초,4.9,0.0,NaN,...,1200.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,28.25
1,7105,7105,1,2004010101,2,90,속초,4.9,0.0,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,27.70
2,7106,7106,2,2004010102,3,90,속초,4.2,0.0,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,28.77
3,7107,7107,3,2004010103,4,90,속초,3.3,0.0,NaN,...,1800.0,0.0,NaN,-0.9,0.0,NaN,NaN,NaN,NaN,30.13
4,7108,7108,4,2004010104,5,90,속초,4.1,0.0,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,29.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5493089,5500193,5500193,791626,2011123119,8756,295,남해,3.7,0.0,NaN,...,NaN,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,36.68
5493090,5500194,5500194,791627,2011123120,8757,295,남해,1.6,0.0,NaN,...,NaN,NaN,NaN,-0.4,0.0,NaN,NaN,NaN,NaN,42.43
5493091,5500195,5500195,791628,2011123121,8758,295,남해,0.8,0.0,NaN,...,NaN,NaN,NaN,-0.6,0.0,NaN,NaN,NaN,NaN,44.30
5493092,5500196,5500196,791629,2011123122,8759,295,남해,0.6,0.0,NaN,...,NaN,NaN,NaN,-0.8,0.0,NaN,NaN,NaN,NaN,43.01


In [13]:
occur_place_his['발생일시'] = occur_place_his['발생일시'].map(lambda x : str(x)[:-2])
occur_place_his

C:\Users\yujeyeong\AppData\Local\Temp\ipykernel_53040\3511259944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  occur_place_his['발생일시'] = occur_place_his['발생일시'].map(lambda x : str(x)[:-2])


,발생일시,발생장소(시도),발생장소(시군구),지점명
901,2011062117,경남,창녕,밀양
902,2011062114,충북,옥천,대전
903,2011062113,경남,함안,의령
904,2011062014,경남,산청,산청
905,2011062013,경기,여주,이천
...,...,...,...,...
4004,2004010714,경남,합천,합천
4005,2004010515,서울,관악,서울
4006,2004010315,경북,울진,울진
4007,2004010311,대구,동,대구


In [14]:
df_dict = {}
for i in range(len(occur_place_his)):
    row = occur_place_his.iloc[i]
    df_dict[row['발생일시']+row['지점명']] = 1
df_dict

{'2011062117밀양': 1,
 '2011062114대전': 1,
 '2011062113의령': 1,
 '2011062014산청': 1,
 '2011062013이천': 1,
 '2011061619부여': 1,
 '2011061615수원': 1,
 '2011061214동해': 1,
 '2011052213울산': 1,
 '2011051913대전': 1,
 '2011051414철원': 1,
 '2011050515영광': 1,
 '2011042923홍성': 1,
 '2011042514서산': 1,
 '2011042415봉화': 1,
 '2011042118울산': 1,
 '2011042114함양': 1,
 '2011042112천안': 1,
 '2011042111광주': 1,
 '2011042017남원': 1,
 '2011042016청송': 1,
 '2011042014원주': 1,
 '2011042014정읍': 1,
 '2011042012인천': 1,
 '2011041915전주': 1,
 '2011041913순창': 1,
 '2011041913완도': 1,
 '2011041910홍성': 1,
 '2011041714원주': 1,
 '2011041713충주': 1,
 '2011041712인제': 1,
 '2011041712봉화': 1,
 '2011041712남해': 1,
 '2011041618춘천': 1,
 '2011041414구미': 1,
 '2011041413거창': 1,
 '2011041413울진': 1,
 '2011041323춘천': 1,
 '2011041316영월': 1,
 '2011041316동해': 1,
 '2011041315남원': 1,
 '2011041315속초': 1,
 '2011041314전주': 1,
 '2011041312광주': 1,
 '2011041220홍성': 1,
 '2011041215홍성': 1,
 '2011041215부산': 1,
 '2011041215목포': 1,
 '2011041214동두천': 1,
 '2011041211청주': 1,

In [15]:
def occur_match(tm,name):
    try:
        return df_dict[tm+name]
    except:
        return 0
#     for i in range(len(occur_place_his)):
#         row = list(occur_place_his.iloc[i])
        
#         if (tm == row[0]) & (name == row[3]):
#             #print(tm, row[0], name, row[3])
#             return 1
#     return 0



In [16]:
#라벨링
sample_df_humid['label'] = sample_df_humid.apply(lambda x: occur_match(x['tm'], x['stnNm']),axis=1)
sample_df_humid

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,tm,rnum,stnId,stnNm,ta,taQcflg,rn,...,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te,실효습도,label
0,7104,7104,0,2004010100,1,90,속초,4.9,0.0,NaN,...,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,28.25,0
1,7105,7105,1,2004010101,2,90,속초,4.9,0.0,NaN,...,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,27.70,0
2,7106,7106,2,2004010102,3,90,속초,4.2,0.0,NaN,...,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,28.77,0
3,7107,7107,3,2004010103,4,90,속초,3.3,0.0,NaN,...,0.0,NaN,-0.9,0.0,NaN,NaN,NaN,NaN,30.13,0
4,7108,7108,4,2004010104,5,90,속초,4.1,0.0,NaN,...,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,29.13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5493089,5500193,5500193,791626,2011123119,8756,295,남해,3.7,0.0,NaN,...,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,36.68,0
5493090,5500194,5500194,791627,2011123120,8757,295,남해,1.6,0.0,NaN,...,NaN,NaN,-0.4,0.0,NaN,NaN,NaN,NaN,42.43,0
5493091,5500195,5500195,791628,2011123121,8758,295,남해,0.8,0.0,NaN,...,NaN,NaN,-0.6,0.0,NaN,NaN,NaN,NaN,44.30,0
5493092,5500196,5500196,791629,2011123122,8759,295,남해,0.6,0.0,NaN,...,NaN,NaN,-0.8,0.0,NaN,NaN,NaN,NaN,43.01,0


In [53]:
sample_df_humid[(sample_df_humid['tm']=='2004010311') & (sample_df_humid['stnNm']=='대구')]

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,tm,rnum,stnId,stnNm,ta,taQcflg,rn,...,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te,실효습도,label
218867,225971,225971,218867,2004010311,60,143,대구,5.8,0.0,NaN,...,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,39.14,0


In [29]:
list(occur_place_his.iloc[1])[0]

'2011062114'

In [39]:
sample_df_humid.iloc[385718]['tm']

'2004010114'

In [42]:
if (sample_df_humid.iloc[385718]['tm'] =='2004010114') & (sample_df_humid.iloc[385718]['stnNm'] == '인제'):
    print('x')

x


In [19]:
sample_df_humid[sample_df_humid['label']==1]
final_sample_table=sample_df_humid[["tm","stnNm",'ta','rn','ws','wd','hm','실효습도','label']]
final_sample_table['tm'] = final_sample_table['tm'].map(lambda x:x+'00')
final_sample_table

C:\Users\yujeyeong\AppData\Local\Temp\ipykernel_53040\3662262653.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_sample_table['tm'] = final_sample_table['tm'].map(lambda x:x+'00')


,tm,stnNm,ta,rn,ws,wd,hm,실효습도,label
0,200401010000,속초,4.9,NaN,6.3,230.0,39.0,28.25,0
1,200401010100,속초,4.9,NaN,4.8,250.0,36.0,27.70,0
2,200401010200,속초,4.2,NaN,3.0,250.0,35.0,28.77,0
3,200401010300,속초,3.3,NaN,2.4,270.0,40.0,30.13,0
4,200401010400,속초,4.1,NaN,2.8,270.0,36.0,29.13,0
...,...,...,...,...,...,...,...,...,...
5493089,201112311900,남해,3.7,NaN,1.3,270.0,46.0,36.68,0
5493090,201112312000,남해,1.6,NaN,0.5,270.0,54.0,42.43,0
5493091,201112312100,남해,0.8,NaN,0.7,250.0,57.0,44.30,0
5493092,201112312200,남해,0.6,NaN,0.7,230.0,60.0,43.01,0


In [21]:
final_sample_table.to_csv('final_training_data_2004_2011.csv')